Data/library imports

In [105]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

project = pd.read_csv('../part1/Project_data.csv')
print(project.shape)
project.head()

(1564, 42)


,Obs_No,AR #,Industry,Obs_Year,Year Live,DaysSinceLive,Reported Number of Employees,EE Year Count,ARR Year Growth,ARR Start,...,Category_Account Health,RefNo_Cumulative,Connections Attendance,Training Hours,Training Students,Percent of Book,ACT Score,BI Score,Ave P_Score,Model
0,1,ABE1000,Wholesale Trade,0,2011,1823,222,217,4753.0,66054,...,1,0,0,0.0,0,1.172813,103.600918,8.841379,118.166034,1
1,2,ABE1000,Wholesale Trade,1,2011,2188,222,218,5866.0,70807,...,1,0,0,0.0,0,1.099129,103.108676,3.775498,73.553312,1
2,3,ABI1002,"Technology, Professional, and Business Services",1,2016,240,648,381,39173.0,73369,...,1,0,0,0.0,0,2.492619,52.468248,1.425152,17.787642,1
3,4,ACC1004,"Technology, Professional, and Business Services",0,2010,2261,220,220,3192.0,31944,...,1,0,0,36.0,4,1.714876,9.036163,15.128390,9.036163,1
4,5,ACC1004,"Technology, Professional, and Business Services",1,2010,2626,220,220,3504.0,35136,...,1,0,0,36.0,4,1.559085,30.368201,15.128390,30.368201,1


In [106]:
%matplotlib inline

Dropping columns not in model

In [107]:
project = project.drop("Obs_No", axis="columns")
project = project.drop("AR #", axis="columns")
project = project.drop("Obs_Year", axis="columns")
project = project.drop("Year Live", axis="columns")
project = project.drop("Reported Number of Employees", axis="columns")
project = project.drop("ARR_EndOfYear", axis="columns")
project = project.drop("RefNo_Cumulative", axis="columns")
project = project.drop("BumpOut75K", axis="columns")
project = project.drop("growth_percent", axis="columns")

In [108]:
dummies=pd.get_dummies(project, columns=["Industry ", "Category_Account Health", "Connections Attendance"])
dummies.columns

Index(['DaysSinceLive', 'EE Year Count', 'ARR Year Growth', 'ARR Start',
       'Product 1', 'Product 2 ', 'Product 3', 'Product 4', 'Product 5',
       'Product 6', 'Product 7', 'Product 8', 'Product 9', 'Product 10',
       'Product 11', 'Product 12', 'Product 13', 'Product 14', 'Product 15',
       'Product 16', 'Product 17', 'Product 18', 'Total Product Count',
       'Training Hours', 'Training Students', 'Percent of Book', 'ACT Score',
       'BI Score', 'Ave P_Score', 'Model ',
       'Industry _Agriculture and Natural Resources', 'Industry _Construction',
       'Industry _Education', 'Industry _Finance and Insurance',
       'Industry _Healthcare', 'Industry _Manufacturing',
       'Industry _Non-Profits, Membership, and Human Services',
       'Industry _Public Sector', 'Industry _Real Estate',
       'Industry _Restaurants and Hospitality', 'Industry _Retail',
       'Industry _Sports and Entertainment',
       'Industry _Technology, Professional, and Business Services',
   

In [109]:
dummies=dummies.drop("Category_Account Health_1", axis ="columns")
dummies=dummies.drop("Industry _Technology, Professional, and Business Services", axis ="columns")
dummies=dummies.drop("Connections Attendance_0", axis ="columns")
dummies.columns

Index(['DaysSinceLive', 'EE Year Count', 'ARR Year Growth', 'ARR Start',
       'Product 1', 'Product 2 ', 'Product 3', 'Product 4', 'Product 5',
       'Product 6', 'Product 7', 'Product 8', 'Product 9', 'Product 10',
       'Product 11', 'Product 12', 'Product 13', 'Product 14', 'Product 15',
       'Product 16', 'Product 17', 'Product 18', 'Total Product Count',
       'Training Hours', 'Training Students', 'Percent of Book', 'ACT Score',
       'BI Score', 'Ave P_Score', 'Model ',
       'Industry _Agriculture and Natural Resources', 'Industry _Construction',
       'Industry _Education', 'Industry _Finance and Insurance',
       'Industry _Healthcare', 'Industry _Manufacturing',
       'Industry _Non-Profits, Membership, and Human Services',
       'Industry _Public Sector', 'Industry _Real Estate',
       'Industry _Restaurants and Hospitality', 'Industry _Retail',
       'Industry _Sports and Entertainment',
       'Industry _Transportation and Warehousing',
       'Industry _Ut

Split between Model(train, test) data and data that I will only use for prediction (Z)

In [110]:
pred_data= dummies.loc[:, "Model "]==0
Z=dummies.loc[pred_data, :]
#Z.head()
print(Z.shape)

(465, 58)


In [111]:
## only use rows that have a target value (2018 and 2017 data)
model_data = dummies.loc[:,"Model "]==1
dummies = dummies.loc[model_data, :]
print(dummies.shape)

(1099, 58)


In [7]:
#project.columns
#project.isnull().sum()

FINAL MODEL: RandomForestRegressor

In [113]:
target_col= "ARR Year Growth"

X= dummies.drop([target_col, "Model "], axis = "columns")
y=dummies.loc[:, target_col]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

from sklearn.ensemble import RandomForestRegressor

rfr=RandomForestRegressor(n_estimators=100, max_depth=3, oob_score=True)

rfr.fit(X_train, y_train)

print(rfr.score(X_train, y_train))
print(rfr.score(X_test, y_test))

0.39806284065915576
0.27006083180503626


In [114]:
rfr.feature_importances_

array([4.13805270e-03, 2.21893152e-02, 2.89809268e-01, 0.00000000e+00,
       1.48799295e-02, 0.00000000e+00, 6.79889823e-03, 1.64280862e-03,
       5.47324269e-04, 4.62187042e-04, 3.43557659e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.99204902e-03,
       0.00000000e+00, 0.00000000e+00, 3.04792796e-02, 2.46150895e-03,
       0.00000000e+00, 1.22795246e-02, 1.79175644e-01, 1.86564859e-01,
       2.85540984e-02, 1.38665914e-01, 1.65251893e-02, 1.37858331e-02,
       0.00000000e+00, 6.54603524e-04, 0.00000000e+00, 1.09933706e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.98396157e-03, 0.00000000e+00, 2.07003891e-02, 5.53766976e-04,
       0.00000000e+00, 6.31696553e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.31903613e-04,
       8.44097245e-04, 5.25700580e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [115]:
feature_imp=list(zip(dummies
         .drop(["ARR Year Growth", "Model "], axis='columns'),
    rfr.feature_importances_)
)
feature_imp

[('DaysSinceLive', 0.004138052702439954),
 ('EE Year Count', 0.02218931520880983),
 ('ARR Start', 0.2898092675791911),
 ('Product 1', 0.0),
 ('Product 2 ', 0.014879929504610501),
 ('Product 3', 0.0),
 ('Product 4', 0.006798898226081549),
 ('Product 5', 0.0016428086224387556),
 ('Product 6', 0.0005473242694988632),
 ('Product 7', 0.00046218704190593366),
 ('Product 8', 0.0003435576590667841),
 ('Product 9', 0.0),
 ('Product 10', 0.0),
 ('Product 11', 0.0),
 ('Product 12', 0.0),
 ('Product 13', 0.004992049023683243),
 ('Product 14', 0.0),
 ('Product 15', 0.0),
 ('Product 16', 0.030479279610763122),
 ('Product 17', 0.002461508950740288),
 ('Product 18', 0.0),
 ('Total Product Count', 0.012279524607262844),
 ('Training Hours', 0.17917564366444938),
 ('Training Students', 0.18656485944581291),
 ('Percent of Book', 0.028554098386920298),
 ('ACT Score', 0.13866591445926454),
 ('BI Score', 0.016525189257871026),
 ('Ave P_Score', 0.01378583305281159),
 ('Industry _Agriculture and Natural Resour

In [116]:
rfr.oob_score_

0.22297233798165716

In [72]:
#Z.shape

(465, 57)

In [117]:
Z=Z.drop(["ARR Year Growth", "Model "], axis ="columns")

In [118]:
Z_pred = rfr.predict(Z)
Z_pred

array([  2836.68910308,   2676.86446292, -14101.13884333,  -6028.46796666,
        -6395.42429842,   2308.99774064,   2607.47787184,   2836.68910308,
         2807.25192646,   2711.01482866,   2444.41180667,   2338.79451629,
         2807.25192646, -14589.5946506 ,   2836.68910308,  -6170.43213812,
         2343.14810638,   2836.68910308,   2116.38210943,   -961.98412079,
         2802.53873734,   2836.68910308,   2711.01482866,   2343.14810638,
         4531.38742188,   2239.61114955,  -1818.22614241,   2711.01482866,
         2711.01482866,   2836.68910308,   2773.10156071,   2836.68910308,
       -15315.96592312,   2777.21958847,  -4969.53997218,   2836.68910308,
       -12556.20071857,   2711.01482866,   2836.68910308,   2607.47787184,
       -13536.42728941,   2711.01482866,   2711.01482866,   2836.68910308,
         2759.84220024,   2673.68263165,   2836.68910308,  -4788.75569021,
         2836.68910308,   2711.01482866,   2343.14810638,   2836.68910308,
         2711.01482866,  

In [119]:
Z_pred.sum()

335979.80018931604

In [120]:
Z.loc[:, 'predictions'] = Z_pred

In [121]:
Z

,DaysSinceLive,EE Year Count,ARR Start,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,...,Connections Attendance_1,Connections Attendance_2,Connections Attendance_3,Connections Attendance_4,Connections Attendance_5,Connections Attendance_6,Connections Attendance_7,Connections Attendance_8,Connections Attendance_12,predictions
1099,2991,220,38640,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2836.689103
1100,2916,177,67320,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2676.864463
1101,2741,230,74989,1,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,0,-14101.138843
1102,2371,271,71075,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,-6028.467967
1103,5086,1160,70853,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,-6395.424298
1104,3470,288,66486,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2308.997741
1105,1006,260,63804,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,2607.477872
1106,544,97,25800,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2836.689103
1107,2735,42,20700,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2807.251926
1108,2833,301,65666,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2711.014829


In [71]:
Z.to_csv('out.csv')